## Stargate SG-1: Cute and Fuzzy
#### Your Mission
Given a string containing the current state of the control crystals inner pathways (labeled as "X") and its gaps (labeled as "."), generate the shortest path from the start node (labeled as "S") to the goal node (labeled as "G") and return the new pathway (labeled with "P" characters).  
If no solution is possible, return the string "Oh for crying out loud..." (in frustration).  

#### The Rules
* Nodes labeled as "X" are not traversable.
* Nodes labeled as "." are traversable.
* A pathway can be grown in eight directions (up, down, left, right, up-left, up-right, down-left, down-right), so diagonals are possible.
* Nodes labeled "S" and "G" are not to be replaced with "P" in the case of a solution.
* The shortest path is defined as the path with the shortest euclidiean distance going from one node to the next.
* If several paths are possible with the same shortest distance, return any one of them.  

Note that the mazes won't always be squares.
```
.S...             .SP..
XXX..             XXXP.
.X.XX      =>     .XPXX
..X..             .PX..
G...X             G...X
```

In [1]:
import collections

Pos = collections.namedtuple('Pos', ['y', 'x'])

class Maze:
    def __init__(self, orig):
        self.i_steps = [ (0,1), (1,1), (1,0), (1,-1), (0,-1), (-1,-1), (-1,0), (-1,1) ]
        self.orig = orig
        self.make_maze()
        self.g_dist = 2 ** 0.5
        
    def make_maze(self):
        self.y_size = len(self.orig)
        self.x_size = len(self.orig[0])
        self.maze = [ [0]*len(l) for l in self.orig ]
        for y in range(self.y_size):
            for x in range(self.x_size):
                c = self.orig[y][x]
                if c == 'X': 
                    self.maze[y][x] = -1
                elif  c == 'S':
                    self.s = Pos(y, x)
                elif  c == 'G':
                    self.g = Pos(y, x)
                    
    def is_valid_next_pos(self, p, d): 
        return (
            0 <= p.x < self.x_size and 0 <= p.y < self.y_size and
               ( self.maze[p.y][p.x] == 0 or self.maze[p.y][p.x] > d )
        )
    
    def distance(self, p1, p2): 
        """calculate euclidiean distance between 2 neigbours """
        return 1 if p1.y == p2.y or p1.x == p2.x else self.g_dist

    def is_valid_back_pos(self, p): 
        return 0 <= p.x < self.x_size and 0 <= p.y < self.y_size and self.maze[p.y][p.x] > 0
        
    def process_current_steps(self, queue):
        stack = collections.deque(maxlen=len(queue)*7)
        for p in queue:
            for i in self.i_steps:
                next_p = Pos(p.y + i[0], p.x + i[1])
                next_p_d = self.maze[p.y][p.x] + self.distance(p, next_p)
                if self.is_valid_next_pos(next_p, next_p_d):
                    self.maze[next_p.y][next_p.x] = next_p_d
                    if next_p.y == self.g.y and next_p.x == self.g.x: continue
                    stack.append(next_p)
        return stack

    def find_back_smallest_step(self, p):
        """Find shortest path step back"""
        steps = filter(self.is_valid_back_pos, [ Pos(p.y + i[0], p.x + i[1]) for i in self.i_steps ])
        return min(steps, key = lambda s: self.maze[s.y][s.x] + self.distance(s, p))

    def find_shortest_back_path(self):
        """Find shortest path back"""
        p = self.g
        while not(p.y == self.s.y and p.x == self.s.x):
            p = self.find_back_smallest_step(p)
            self.orig[p.y][p.x] = 'P'
        self.orig[self.s.y][self.s.x] = 'S'
        return self.orig
    
    def get_path_distance(self): return self.maze[self.g.y][self.g.x] - 1
    
    def get_path_length(self):
        return 1 + len(list(filter( lambda c: c =='P', "\n".join([ "".join(l) for l in self.orig ]))))
                    
    def find_path(self):
        stack = collections.deque(maxlen=1)
        stack.append(self.s)
        self.maze[self.s.y][self.s.x] = 1
        while len(stack) > 0: stack = self.process_current_steps(stack)
        return self.find_shortest_back_path() if  self.maze[self.g.y][self.g.x] > 0 else False


def wire_DHD_SG1(existingWires):
    m = [ list(l) for l in existingWires.split('\n') ]
    maze = Maze(m)
    r = maze.find_path()
    if r:
        print("Path distance: {}, Path Length: {}".format(maze.get_path_distance(), maze.get_path_length()))
        return [ "".join(l) for l in r ] #"\n".join([ "".join(l) for l in r ])
    return "Oh for crying out loud..."    

In [2]:
s = "...\nS.G\n..."
print(s)
wire_DHD_SG1(s)


...
S.G
...
Path distance: 2, Path Length: 2


['...', 'SPG', '...']

In [3]:
s = existingWires = "...\n.S.\n...\n.G.\n..."
print(s)
wire_DHD_SG1(s)

...
.S.
...
.G.
...
Path distance: 2, Path Length: 2


['...', '.S.', '.P.', '.G.', '...']

In [4]:
s = """
SX.
XX.
..G
""".strip('\n')

wire_DHD_SG1(s)

'Oh for crying out loud...'

In [5]:
s = existingWires = """
SX.
X..
XXG
""".strip('\n')
wire_DHD_SG1(s)

Path distance: 2.82842712474619, Path Length: 2


['SX.', 'XP.', 'XXG']

In [6]:
s = """
.X.X.X....XXXXXX...X
XX.XX.XXXXXXXXXXX..X
.X.X.XX..X..X.XXXXXX
X.X..XXX...XX.X.XXX.
X.X..X..XXX.X.X.X...
.XXX..XXXXX.X.X..XX.
X.XX.SX......XXX..X.
.XXXXX.XXX...XX..X..
....X.XX..X.XX.X..XX
....X..XX..XX..X.XX.
X...X..XX.X.X.XX...X
.XXX.........X.XX..G
..XX.XX.XX.X.XXXXXX.
.X.X...X.X.XXXX..X.X
..X..XXX.XX....XXXX.
XX..XXXXXXX.....XXXX
XXXX.X.X..XXXXXX...X
X...X..X..XXXX..X..X
X.XXXXX..XX..XXX.X.X
XX.X.XX.XXXX.X..X.XX
""".strip('\n')

wire_DHD_SG1(s)

Path distance: 23.142135623730955, Path Length: 19


['.X.X.X....XXXXXX...X',
 'XX.XX.XXXXXXXXXXX..X',
 '.X.X.XX..X..X.XXXXXX',
 'X.X..XXX...XX.X.XXX.',
 'X.X..X..XXX.X.X.X...',
 '.XXX..XXXXX.X.X..XX.',
 'X.XX.SX......XXX..X.',
 '.XXXXXPXXX...XXP.X..',
 '....XPXX..X.XXPXP.XX',
 '....XP.XX..XX.PXPXX.',
 'X...X.PXX.X.XPXX.P.X',
 '.XXX...PPPPPPX.XX.PG',
 '..XX.XX.XX.X.XXXXXX.',
 '.X.X...X.X.XXXX..X.X',
 '..X..XXX.XX....XXXX.',
 'XX..XXXXXXX.....XXXX',
 'XXXX.X.X..XXXXXX...X',
 'X...X..X..XXXX..X..X',
 'X.XXXXX..XX..XXX.X.X',
 'XX.X.XX.XXXX.X..X.XX']